In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import viz

COLORS, PAGE_WIDTH, ROW_HEIGHT = viz.set_plots()


def rolling_mean(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode="valid")

In [ ]:
prices = pd.read_csv("../data/prices.csv", index_col=0, parse_dates=True)
prices2023 = prices.loc[f"2023-01-01":f"2023-01-31"].lmp

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(PAGE_WIDTH, ROW_HEIGHT))
prices2023.plot(ax=ax)
ax.set_title("2023 LMP")
ax.set_ylabel("LMP ($/MWh)")
plt.show()

In [ ]:
# from sacRun import main as sacRun
# sacRun(nYears=None, nMonths = 5, month= [1], test = 1, ep=800)

In [ ]:
scores = np.load("SACResults/scores.npy")

f, ax = plt.subplots(1, 1, figsize=(PAGE_WIDTH, ROW_HEIGHT))
ax.plot(scores)
ax.plot(rolling_mean(scores, 10))
ax.set_title("SAC Scores")
ax.set_ylabel("Score")
plt.show()

In [ ]:
history = pd.read_csv("SACResults/sac_history.csv", index_col=0, parse_dates=True)

START, END = "2023-01-01", "2023-01-25"

viz.plotOperations(history, START, END, save_path="SACResults/policy.png")

In [ ]:
from softActorCritic import SAC

from tqdm import tqdm

rewards = []
max_power = []
for _ in tqdm(range(1000)):
    PolicyPPO = SAC([])
    pricesTest = pd.concat([prices2023[-4:], prices2023])
    history = PolicyPPO.test(pricesTest)
    rewards.append(history.reward.sum())
    max_power.append(history.load.max())
    # print(history.reward.sum())

print("Reward Metrics: ", np.mean(rewards), "$")
print("Max Power Metrics: ", np.mean(max_power), "MW")

In [ ]:
print("Std Reward Metrics: ", np.std(rewards), "$")
print("Std Max Power Metrics: ", np.std(max_power), "MW")